<a href="https://colab.research.google.com/github/sera0911/asia-ai-study/blob/main/Python_web_programming_day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### day2 정리  

1. 모델 클래스 (models.Model 상속) 정의  
2. DB에 모델 클래스에 대한 매핑 테이블 생성 :  
 makemigrations로 테이블 작업 파일 생성  
  migrate로 테이블 생성  

3. 장고 셸 기반으로 insert 수행(Model 클래스 생성자)  

```
  조회(select) -  Model 클래스.objects.all(),filter(),get()  
  수정(update) - Model 클래스.objects.get()로 반환된 객체.속성(컬럼) = 값  
  삭제(delete) - Model 클래스.objects.get() 로 반환된 객체.delete()  
```

장고 Admin기반으로 모델 CRUD 수행  
1. super user 생성 (python manage.py createsuperuser 명령)  
2. 장고프로젝트폴더/admin.py에 admin.site.register(Question) 모델 클래스 등록  
3. 데이터 검색기능 장고프로젝트폴더/admin.py에 추가  

```
  class QuestionAdmin(admin.ModelAdmin):
      search_fields = ['subject']

 등록=> admin.site.register(...QustionAdmin)  
```

질문 목록 데이터를 template으로 서비스  
1. 앱폴더/urls.py파일에 요청 url과 view함수 매핑 선언  
2. 앱폴더/views.py 파일에 view함수 정의 (index(request), return render(request, 템플릿이름, 파라미터 )

템플릿 설정  
 템플릿 디렉터리를 장고 config/settings.py 파일에 등록  

```
TEMPLATES = [
   {
    ...
    'DIRS': [BASE_DIR / 'templates'],
    ...
   }
]
```

앱폴더/templates 디렉토리 생성  
3. 템플릿 파일 생성 (앱폴더/templates/question_list.html)   

템플릿 태그  

 
 ```
  {% if 조건 %}  
      html내용(태그)
   {% elif 조건 %}
      html내용(태그)
  {% else %}
      html내용(태그)
    {% endif %}
      
  {% for 변수 in 집합객체 %}  
    html내용(태그)  {{forloop.counter 또는 forloop.counter0}}  
  {% endfor %}  

  {{출력내용(변수, 객체.속성, 반환값있는 함수}}  
```

[질문 목록에서 제목을 클릭하면 pk값을 상세 페이지 서비스]  
question_list페이지에서 글 제목 클릭(a태그, 요청url과 글번호파라미터 전달)   

상세 페이지 요청 url이 앱폴더/urls.py 에서 view함수 검색 (question_detail)  

앱폴더/views.py에서 view함수 찾아서 실행   (model객체 사용해서 조회, render함수로 템플릿 페이지 정의)  

질문 상세 템플릿 페이지 생성(앱폴더/templates/question_detail.html)   

DoseNotExist오류 화면의 오류 처리를 위해서 get_object_or_404(객체, pk=) 
 


### day3 start   
### 목표: 질문 작성할 수 있는 웹페이지를 제공  , 답변을Model객체를 사용해서 DB에 저장  

- <int:question_id>

http://server(ip 또는 dns): 8000/first/글번호 = int:question_id로 나타낸 것 (실제 주소 대신에 url 요청 주소 패턴이 변결되어도 유지보수가 용이하도록 별칭을 사용할 수 있다)  

#### 템플릿의 href에 실제 주소가 아니라 url 별칭을 사용하려면 urls.py파일을 수정하고, path함수에 있는 url매핑에 name속성을 설정한다  

1. 앱폴더/urls.py 파일에 name설정 

```
...
path('', views.index, name='index' ),
    path('<int:question_id>/', views.detail, name='detail' ), 
```

2. 앱폴더/*.html 템플릿에서 앵커 태그 url 실제 주소가 아니라, 별칭 사용  

question_list에서 수정

```
{% if question_list %}
     <ul>
        {% for question in question_list %}
         <!-- 주석처리 한 것
         <li> <a href="/first/{{ question.id }}/">{{question.subject}}</a></li>
         -->
         <li> <a href="{% url 'detail' question.id %}">{{question.subject}} </a> </li>  #새로 적은 코드
        {% endfor %}
```

#### 앱이 여러개 구성되면 url 별칭에 대해 충돌이 발생하므로 url 별칭을 앱.별칭으로 사용하면 충돌 예방하기 위해 각각의 앱이 관리하는 독립적인 이름 공간을 가지도록 namespace 개념 도입  

1. 앱폴더/urls.py 파일에 app_name='url 네임스페이스' 설정  

urls.py에서 추가
```
app_name = 'first'
```

2. 앱폴더/*.html 템플릿에서 앵커 태그의 템플릿 태그내에 url에 url 네임스페이스: 별칭 (firtst:detail)

question_list 수정

```
<li> <a href="{% url 'first:detail' question.id %}">{{question.subject}} </a> </li>
```


#### 질문 작성할 수 있는 웹페이지를 제공  ,답변을Model객체를 사용해서 DB에 저장   

1. 질문 상세 템플릿에 답변 등록 버튼 추가 

question_detail.html에 추가
```
<form  action="{% url 'first:answer_create' question.id %}"  method="post" > #first라는 네임스페이스 지정, 주로 post로 요청을 보냄
    {% csrf_token %}  #템플릿 태그
    <textarea  name="content" id="content" rows="20"></textarea>  #name= 실제 파라미터로 넘길 때 이름이 된다, id와 같은 이름을 하는 것이 매핑에 좋다
    <input type="submit" value="답변등록">
</form>
```
- form 태그 바로 밑에 {% csrf_token %}를 항상 입력  
- {% csrf_token %} 는 form 엘리먼트를 통해 전송된 데이터(답변)가 실제로 웹 브라우저에서 작성된 페이지인지 판단하는 검사기 역할을 한다  

- 해킹을 통해서 form 작성 데이터가 아닌 다른 데이터를 전송하게 되면 브라우저가 보낸 csrf_token값과 해커 보낸 값이 일치하지 않으면 오류를 발생시켜 보안을 유지할 수 있도록 해줌  

- 장고 프레임 워크에 CsrfViewMiddleware 라는 미들웨어가 추가되어 있어야만 사용 가능  


2. 앱폴더/urls.py파일에 답변 등록을 위한 url 매핑 설정

urls.py에서 추가

```
urlpatterns = [
  ...

path('answer/create/<int:question_id>/', views.answer_create, name='answer_create' ),
```

3. 앱폴더/views.py에 함수 정의 

views.py에 함수 추가
```
def answer_create(request, question_id) :
    """
    답변 내용 등록
    """
    question = get_object_or_404(Question, pk=question_id)
    question.answer_set.create(content=request.POST.get('content') , create_date=timezone.now())
    #Answer 모델이 Question 모델을 Foreign Key로 참조하고 있으므로, question객체.answer_set으로 사용할 수 있다
```




목표: 웹 페이지에 디자인을 적용하려면 스타일시트(CSS) 적용  
1. config/settings.py 파일에 STATICFILESE_DIRS에 디렉토리 경로를 추가

```
STATICFILES_DIRS = [
    BASE_DIR / 'static',
]
```
스타일 시트를 장고앱에 적용하려면 CSS 파일이 static 디렉토리에 있어야 한다  

2. static 디렉토리 만들고 스타일 시트 지정  (예시로 한 것이라서 테스트용으로만 사용)  

장고프로젝트폴더/static 디렉토리 생성
style.css 파일을 만들어 추가

```
textarea {
   width:100%;
}

input[type=submit] {
    margin-top:10px;
 }
```

3. 템플릿에 스타일 적용

question_detail.html에 맨 위에 추가
```
{% load static %}
<link rel="stylesheet" type="text/css" href="{% static 'style.css' %}" >
```



- 부트스트랩으로 화면 꾸미기  

부트스트랩은 트위터를 개발하면서 화면 디자인 작업을 위해 만들어졌고 지속적으로 관리 되고 있는 오픈소스 프로젝트이다.  

- 부트스트랩 다운로드 후 압출 풀어 bootstrap.min.css 파일만 복사해서 장고프로젝트폴더/static 폴더에 저장
-템플릿에 스타일 적용   

question_list.html
```
<link rel="stylesheet" type="text/css" href="{% static 'bootstrap.min.css' %}" >
```

템플릿 상속 기능 
1. 모든 템플릿에서 공통적으로 입력할 내용을 포함할 기본 틀인 base.html 템플릿을 만듬 
 
templates 폴더 아래 base.html 만들기  

```
<!DOCTYPE html> <!--문서 유형 선언 -->
<html lang="ko">
<head>
    {% load static %}
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width,initial-scale=1, shrink-to-fit=no">
    <link rel="stylesheet" type="text/css" href="{% static 'bootstrap.min.css' %}">
    <title>장고 first 앱 </title>
</head>
<body>
{% block content %}
{% endblock %}
</body>
</html>
```

detail, list에도 위아래 추가

```
{% extends 'base.html' %}
{% block content %}  
...
{% endblock %}
```



질문 등록 기능 구현  
1. 질문 등록 버튼 추가 (question_list.html)  
```
<a href="{% url 'first:question_create' %}" class="btn btn-primary"> 질문 등록하기</a>
```

2. 앱폴더/urls.py 파일에 질문 등록을 위한 요청 url과 뷰함수 매핑 등록  

```
path('question/create/', views.question_create, name='question_create' ),
```
3. 앱폴더/views.py에 함수 정의 (요청 처리 후 render 또는 redirect로 응답)

views.py
```
def question_create(request):
    """
    질문 내용 등록
    """
    if request.method == 'POST':
        form = QuestionForm(request.POST)
        if form.is_valid():
            question = form.save(commit=False)   #메모리에  임시 저장
            question.create_date = timezone.now()
            question.save()     #DB에 Commit 저장
            return redirect('first:index')
    else :               #GET 방식
        form = QuestionForm()

    context = {'form': form}
    return render(request, 'first/question_form.html', context)
```

forms.ModelForm을 상속받아 장고 Form을 render함수에 {'form':form} 인수로 지정하여 폼 엘리먼트 생성

4. 장고 폼 클래스 생성  

forms.ModelForm을 상속받는 QuestionForm클래스 생성  
내부 클래스로 Meta클래스 정의(model, fields)  
Meta클래스에는 모델 폼이 사용할 모델 클래스와 모델의 필드들을 구성한다

5. 템플릿 파일 생성 

templates/first/question_form.html  

{{form.as_p }} 는 폼과 연결된 입력 항목을 입력 할 수 있는 HTML코드를 자동으로 만들어 준다

```
{% extends 'base.html' %}
{% block content %}
<div class="container">
    <h5 class="my-3 border-bottom pb-2">질문등록</h5>
    <form method="post" class="post-form my-3">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit" class="btn btn-primary">저장하기</button>
    </form>
</div>
{% endblock %}
```

질문 내용 (DB) 저장 기능 구현
1. question_form.html 페이지에서 저장버튼 요청 url 설정   

(form 태그에 action이 없으면 요청 url은 현재 페이지의 url로 요청이 전달됨)

```
<button type="submit" class="btn btn-primary">저장하기</button>
```

(앱폴더/urls.py 파일에 질문 저장을 위한 요청 url과 뷰함수 매핑 선언)  

2. 앱폴더/views.py에 함수 정의 (요청 처리 후 render 또는 redirect로 응답)  

get방식의 경우 QuestionForm()과 같이 입력값 없이 객체를 생성  
post방식의 경우 QuestionForm(request.POST)처럼 화면에서 전달받은 데이터로 폼의 값이 채워지도록 객체를 생성  
form.is.valid()는 post 요청으로 받은 form이 유효한지 검사  
(폼이 유효하지 않다면 폼에 오류가 저장되어 화면에 전달됨)  
form.save(commit=False)의 commit=false는 임시저장을 의미  
(실제 데이터는 아직 DB에 저장되지 않은 상태)   
create_date에 값을 설정한 후 question.save()로 실제 DB에 저장

```
def question_create(request):
    """
    질문 내용 등록
    """
    if request.method == 'POST':
        form = QuestionForm(request.POST)
        if form.is_valid():
            question = form.save(commit=False)   #메모리에  임시 저장
            question.create_date = timezone.now()
            question.save()     #DB에 Commit 저장
            return redirect('first:index')
    else :               #GET 방식
        form = QuestionForm()

    context = {'form': form}
    return render(request, 'first/question_form.html', context)
```

{{ form.as_p }} 는 폼과 연결된 입력 항목을 입력할 수 있는 HTML 코드가 자동으로 만들어진 경우, 스타일을 적용하려면 QuestionForm 클래스 내부에 있는 Meta클래스에 widgets 속성을 추가해서 스타일을 설정해야 함  
forms.py에서 수정
```
class QuestionForm(forms.ModelForm):
    class Meta:
        model = Question
        fields = ['subject', 'content']
        labels = {
            'subject': '제목',
            'content': '내용',
        }
        widgets = {
            'subject': forms.TextInput(attrs={'class': 'form-control'}),
            'content': forms.Textarea(attrs={'class': 'form-control', 'rows': 10})
        }
```



부트스트랩은 Desktop, Tablet, Mobile등 화면 크기가 작은 기기를 고려한 '반응형 웹 컨텐츠'까지 적용할 수 있다  
nav 바의 햄버거 메뉴 버튼을 제대로 사용하려면 부트스트랩 자바 스트립트 파일과 제이쿼리 파일을 적용해야 함  
base.html에서 추가  
```
<!DOCTYPE html> <!--문서 유형 선언 -->
<html lang="ko">
<head>
    ...
#두 가지 min.js파일 적용
    <script src="{% static 'bootstrap.min.js' %}" ></script>
    <script src="{% static 'jquery-3.6.0.min.js' %}" ></script>
    <title>장고 first 앱 </title>
</head>
<body>
#여기서부터 nav추가
<nav class="navbar navbar-expand-lg navbar-light bg-light border-bottom">
    <a class="navbar-brand" href="{% url 'first:index' %}">홈</a>
    <button class="navbar-toggler ml-auto" type="button" data-toggle="collapse" data-target="#navbarNav"
            aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation">
        <span class="navbar-toggler-icon"></span>
    </button>
    <div class="collapse navbar-collapse flex-grow-0" id="navbarNav">
        <ul class="navbar-nav">
            <li class="nav-item ">
                <a class="nav-link" href="#">로그인</a>
            </li>
        </ul>
    </div>
</nav>
...
```




질문 목록 게시판에 페이징 처리 구현  
보통 게시판의 목록 페이지 요청시 요청 페이지 파라미터가 인수로 넘어오지 않으므로 defalut값으로 page 파라미터를 1로 설정한다  

장고 프레임워크에서 제공하는 페이징 구현에 사용하는 클래스 Paginator(목록객체, 페이지 당 목록 개수)를 제공  
Paginator 객체.get page()

```
for i in range(250):
    q = Question(subject=f'테스트 데이터:[{i:03d}]', content='테스트', create_date=timezone.now())
    q.save()
```
로 250개의 데이터 생성  
페이지 나누는 것은 다음 시간에~!